In [2]:
import pandas as pd
import os

In [5]:
# Ensure the directories exist before saving
os.makedirs('../data', exist_ok=True)
os.makedirs('../data', exist_ok=True)

In [6]:
# Load the dataset
file_path = '../data/cricinfo_innings_data.csv'
df = pd.read_csv(file_path)

In [7]:
# Drop unnecessary columns and reset the index
df_cleaned = df.drop(columns=[col for col in df.columns if 'Unnamed' in col or 'Column' in col])

In [8]:
# Convert relevant columns to appropriate data types
# The dates in the dataset are in 'YYYY-MM-DD' format, so we let pandas infer it
df_cleaned['Start Date'] = pd.to_datetime(df_cleaned['Start Date'], errors='coerce')  # Convert to datetime
df_cleaned['Score'] = pd.to_numeric(df_cleaned['Score'], errors='coerce')  # Convert Score to numeric

C:\Users\Labib\AppData\Local\Temp\ipykernel_19700\374183996.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_cleaned['Start Date'] = pd.to_datetime(df_cleaned['Start Date'], errors='coerce')  # Convert to datetime


In [ ]:
# Handle missing values
df_cleaned['Result'] = df_cleaned['Result'].fillna('Unknown')
df_cleaned['Target'] = df_cleaned['Target'].fillna(0)
df_cleaned['Lead'] = df_cleaned['Lead'].fillna(0)


In [ ]:
# Remove rows with completely missing values
df_cleaned = df_cleaned.dropna(how='all')

In [ ]:
# Remove duplicates
df_cleaned = df_cleaned.drop_duplicates()

In [ ]:
# Save the cleaned dataset
df_cleaned.to_csv('../data/cricinfo_innings_data_clean.csv', index=False)

In [ ]:
print("Data cleaning completed and saved to '../data/cricinfo_innings_data_clean.csv'")
print("First few rows of the cleaned dataset:")
df_cleaned.head()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load the cleaned dataset
df = pd.read_csv('../data/cricinfo_innings_data_clean.csv')

In [ ]:
# Convert 'Start Date' to datetime again (as it may be read as string from CSV)
df['Start Date'] = pd.to_datetime(df['Start Date'], errors='coerce')

In [ ]:
# Extract Format and Opposition Team from 'Opposition' column
df['Format'] = df['Opposition'].apply(lambda x: x.split(' v ')[0])
df['Opposition Team'] = df['Opposition'].apply(lambda x: x.split(' v ')[1])


In [ ]:
# Create a match-level dataset for result-based analyses
matches = df.groupby(['Start Date', 'Opposition']).first().reset_index()

In [ ]:
# Basic Statistics
print("### Number of Matches by Format")
print(matches['Format'].value_counts())


In [ ]:
print("\n### Number of Matches Against Each Opposition")
print(matches['Opposition Team'].value_counts().head(10))  # Top 10 for brevity


In [ ]:
print("\n### Result Percentages (Match-Level)")
print(matches['Result'].value_counts(normalize=True) * 100)

In [ ]:
# Score Distribution by Format
plt.figure(figsize=(10, 6))
sns.boxplot(x='Format', y='Score', data=df)
plt.title('Score Distribution by Format')
plt.xlabel('Format')
plt.ylabel('Score')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Average Score Over Time
df['Year'] = df['Start Date'].dt.year
avg_score_by_year = df.groupby('Year')['Score'].mean()
plt.figure(figsize=(12, 6))
avg_score_by_year.plot()
plt.title('Average Score Over Time (Per Innings)')
plt.xlabel('Year')
plt.ylabel('Average Score')
plt.grid(True)
plt.show()

In [ ]:
# Average RPO Over Time by Format
avg_rpo_by_year_format = df.groupby(['Year', 'Format'])['RPO'].mean().unstack()
plt.figure(figsize=(12, 6))
avg_rpo_by_year_format.plot()
plt.title('Average Runs Per Over (RPO) Over Time by Format')
plt.xlabel('Year')
plt.ylabel('Average RPO')
plt.legend(title='Format')
plt.grid(True)
plt.show()

In [ ]:
# Results by Decade (Match-Level)
matches['Decade'] = (matches['Start Date'].dt.year // 10) * 10
win_loss_by_decade = matches.groupby(['Decade', 'Result']).size().unstack().fillna(0)
plt.figure(figsize=(12, 6))
win_loss_by_decade.plot(kind='bar', stacked=True)
plt.title('Match Results by Decade')
plt.xlabel('Decade')
plt.ylabel('Number of Matches')
plt.legend(title='Result')
plt.show()

In [ ]:
# Performance Against Oppositions (Match-Level)
win_loss_by_opposition = matches.groupby(['Opposition Team', 'Result']).size().unstack().fillna(0)
win_loss_by_opposition['Win Percentage'] = (win_loss_by_opposition.get('won', 0) / win_loss_by_opposition.sum(axis=1)) * 100
print("\n### Performance Against Top Oppositions (Win Percentage)")
print(win_loss_by_opposition.sort_values('Win Percentage', ascending=False).head(10))


In [ ]:
# Ground Analysis (Match-Level)
top_grounds = matches['Ground'].value_counts().head(10)
print("\n### Top 10 Grounds by Number of Matches")
print(top_grounds)

In [ ]:
# Performance at Top Grounds
df_top_grounds = matches[matches['Ground'].isin(top_grounds.index)]
performance_at_grounds = df_top_grounds.groupby(['Ground', 'Result']).size().unstack().fillna(0)
print("\n### Performance at Top Grounds")
print(performance_at_grounds)